In [92]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from IPython.display import Image


In [93]:
dataset_dir = "/data"
print("Loading dataset from: " + dataset_dir)

Loading dataset from: /data


In [94]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10418548363586635154
xla_global_id: -1
]
Num GPUs Available:  0


In [95]:
image_height = 32
image_width = 32
batch_size = 500

train_dataset = tf.keras.utils.image_dataset_from_directory(
    r"C:/Users/hawry/AI_MH_JG/data/train",
    seed=512,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    r"C:/Users/hawry/AI_MH_JG/data/test",
    seed=512,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

# Print class names
print("Training Classes:")
train_class_names = train_dataset.class_names
print(train_class_names)

print("Validation Classes:")
val_class_names = validation_dataset.class_names
print(val_class_names)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Training Classes:
['FAKE', 'REAL']
Validation Classes:
['FAKE', 'REAL']


In [96]:
def plot_metrics(history, metric):
    plt.plot(history.history[metric], label = metric)
    plt.plot(history.history['val_' + metric], label='val_' + metric)
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend()
    plt.show()

In [97]:
val_true_classes = np.concatenate([y for x, y in validation_dataset], axis=0)
class_names = ['FAKE', 'REAL']
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [98]:
efficientnet_base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(image_height, image_width, 3),
    pooling='max'
)
efficientnet_base_model.trainable = True
inputs = tf.keras.Input(shape=(image_height, image_width, 3))
x = efficientnet_base_model(inputs, training=False)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = Dense(
    256,
    kernel_regularizer=regularizers.l2(0.01),
    activity_regularizer=regularizers.l1(0.01),
    bias_regularizer=regularizers.l1(0.01),
    activation='relu'
)(x)
x = Dropout(rate=0.4, seed=512)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
efficientnet_model = tf.keras.Model(inputs, outputs)

efficientnet_model.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

efficientnet_model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 1280)           │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,268,881 (23.91 MB)

 Trainable params: 6,205,713 (23.67 MB)

 Non-trainable params: 63,168 (246.75 KB)

In [101]:
print("Starting training with Transfer Learning using EfficientNetV2B0...")
efficientnet_model_history = efficientnet_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=40,
    verbose=1,
    callbacks=[early_stopping_callback]
)
print("Transfer Learning training finished.")

Starting training with Transfer Learning using EfficientNetV2B0...
Epoch 1/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 502s 3s/step - accuracy: 0.5038 - loss: 3.9647 - precision_5: 0.5026 - recall_5: 0.8520 - val_accuracy: 0.5049 - val_loss: 3.8821 - val_precision_5: 0.5025 - val_recall_5: 0.9976
Epoch 2/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 518s 3s/step - accuracy: 0.5035 - loss: 3.8552 - precision_5: 0.5025 - recall_5: 0.6824 - val_accuracy: 0.5021 - val_loss: 3.7621 - val_precision_5: 0.5011 - val_recall_5: 0.9988
Epoch 3/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 506s 3s/step - accuracy: 0.5038 - loss: 3.7318 - precision_5: 0.5029 - recall_5: 0.7340 - val_accuracy: 0.5000 - val_loss: 3.6259 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 4/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 507s 3s/step - accuracy: 0.4998 - loss: 3.5874 - precision_5: 0.4759 - recall_5: 0.5153 - val_accuracy: 0.5015 - val_loss: 3.4666 - val_precision_5: 0.5008 - val_recall_5: 0.9999
Epoch 5/40
200/200 ━━━━━━━━━━━━━━━━━━━━ 505s 3s/s

In [1]:
val_loss, val_accuracy, val_precision, val_recall = efficientnet_model.evaluate(validation_dataset)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")

NameError: name 'efficientnet_model' is not defined

In [ ]:
plot_metrics(efficientnet_model_history, 'loss')
plot_metrics(efficientnet_model_history, 'accuracy')
plot_metrics(efficientnet_model_history, 'precision_5')
plot_metrics(efficientnet_model_history, 'recall_5')

In [ ]:
efficientnet_model.save("efficientnet_model.h5")